# SquamataAeroMag - Jupyter notebook for creating metadata for Aeromagnetic data-releases to ScienceBase

### Version 1.8

#### Visit https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroMagEntityAndAttributeArchive.ipynb for Entity and Attribute code from previous Airborne magnetic surveys

This module performs the following operations:
- Create list of data directories.
- Identify files accompanying data release.
- Create file listing for metadata XML markup.
- Identify and load Aeromagnetic *.CSV data file.
- Create file list of data release files
- Create User Editable Keywords Listing
- Create entity and attribute XML markup for all files being released.
- Poplulate metadata template
- Validate metadata; create error log; create HTML and FGDC Text versions of the metadata. (In development - use https://mrdata.usgs.gov/validation/ for validation in the interim)

Known issues needing repair:
- Add items here...

## Future development plans for SquamataSB

- Create all child metadata files from first example created in previous steps. 
- Batch upload files to ScienceBase.
- Batch remove files from ScienceBase. 
- Change ScienceBase parameters such as citation information, add orcid ids, add USGS CMS tags, etc. 

### Instructions
- Create a template xml format that contains boilerplate text common to all childeren in a data release.  Be sure this template contains the approriate curly bracket tags, {SquamataTagExample} used to populate the template using SquamataAeromag.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

Metadata wizard:  Advanced, Open In a jupyter Notebook?
Metadata Wizard 2.o from ScienceBase

v1.0: Read csv File into pandas
v1.1: Create Array of csv column headers, assign header array of channel descriptions and units, create entity and attribute text for csv file
v1.2 Create file listing, create editable keywords text
v1.3 Creat entity and attributes for files other than CSV
v1.4 Populate metadata template
v1.5 Populate variables with boilerplate text, create file lists, write to metadata template.
v1.8 Add standard Order


In [37]:
# Uncomment the below to install sciencebasepy from a directory or unline - be sure to change the path appropriately
##################################################################################################################
#import os, ssl
#import sys

#if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
#    getattr(ssl, '_create_unverified_context', None)): 
#    ssl._create_default_https_context = ssl._create_unverified_context
#    !{sys.executable} -m pip install C:\SBpy\sciencebasepy-master
#################################################################################################################

Processing c:\sbpy\sciencebasepy-master
  Created wheel for sciencebasepy: filename=sciencebasepy-1.6.4-py3-none-any.whl size=16626 sha256=313eea6e624c3099056c181bbd5087b0105fcb1527585ae84f040d80b12e96a8
  Stored in directory: c:\users\pbrown\appdata\local\pip\cache\wheels\7f\34\3f\dfa63aa8cb9976d43735f3c91e76603610f7d543e4c772ae68
Successfully built sciencebasepy
  Attempting uninstall: sciencebasepy
    Found existing installation: sciencebasepy 1.6.4
    Uninstalling sciencebasepy-1.6.4:
      Successfully uninstalled sciencebasepy-1.6.4


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [1]:
# IP-139145
# https://doi.org/10.5066/P9PISI0X
# https://www.sciencebase.gov/catalog/item/623219c9d34ec9f19eeb418b

In [2]:
catNum = '623219c9d34ec9f19eeb418b'

In [3]:
# Phil Brown (pbrown@usgs.gov) 2019 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [4]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [5]:
# Load required Libraries
import sys
import os
import zipfile
import csv
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript
import sciencebasepy as pysb

C:\Users\pbrown\Anaconda3\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


# Collect Information For Metadata Template

In [7]:
# Create an Array of Origin Names between single quotes below
arrOrigin = []

#Change Text Below to the Appropriate Data Release Creators - seperate list items in single quotes ('')
# seperated by a comma (,).
#########################################################################################################################
arrOrigin = ['Grauch, V.J.S.', 'Anderson, E.D.', 'Brown, P.J.'] 
#########################################################################################################################
origin = ''
for i in range(len(arrOrigin)):
    origin = origin + '\n\t\t\t\t<origin>' + arrOrigin[i] + '</origin>'

print ('Origin text:') 
print (origin)    

Origin text:

				<origin>Grauch, V.J.S.</origin>
				<origin>Anderson, E.D.</origin>
				<origin>Brown, P.J.</origin>


###  -- Replace with month, day, year (YYYYMMDD) you request SB page to go live --

In [8]:
#Enter Pub Date between single quotes below

#########################################################################################################################
pubdate = '20220630'
#########################################################################################################################
print (pubdate)

20220630


In [9]:
#Enter Data Release Title between single quotes below

#########################################################################################################################
title = 'Airborne magnetic and radiometric survey of the Wet Mountains and surrounding region, Custer and Fremont Counties, south-central Colorado, 2021'
#########################################################################################################################
print (title)

Airborne magnetic and radiometric survey of the Wet Mountains and surrounding region, Custer and Fremont Counties, south-central Colorado, 2021


In [10]:
#Enter Other Citation between single quotes below

#########################################################################################################################
othercit = 'Additional information about Originators: Tien V.J.S. Grauch, http://orcid.org/0000-0002-0761-3489; Eric D. Anderson, http://orcid.org/0000-0002-0138-6166; Philip J. Brown, http://orcid.org/0000-0002-2415-7462.  Suggested Citation: Grauch, V.J.S., Anderson, E.D., Brown, P.J., 2022, Airborne magnetic and radiometric survey of the Wet Mountains and surrounding region, Custer and Fremont Counties, south-central Colorado, 2021: U.S. Geological Survey data release, https://doi.org/10.5066/P9PISI0X.'
#########################################################################################################################
print (othercit)

Additional information about Originators: Tien V.J.S. Grauch, http://orcid.org/0000-0002-0761-3489; Eric D. Anderson, http://orcid.org/0000-0002-0138-6166; Philip J. Brown, http://orcid.org/0000-0002-2415-7462.  Suggested Citation: Grauch, V.J.S., Anderson, E.D., Brown, P.J., 2022, Airborne magnetic and radiometric survey of the Wet Mountains and surrounding region, Custer and Fremont Counties, south-central Colorado, 2021: U.S. Geological Survey data release, https://doi.org/10.5066/P9PISI0X.


In [11]:
#Enter Online Link between single quotes below

##########################################################################################################################
onlink = 'https://doi.org/10.5066/P9PISI0X'
##########################################################################################################################
print (onlink)

https://doi.org/10.5066/P9PISI0X


### -- Please see minor edits to abstract. Please update the final sentences of the abstract to only refer to the mag data included in this child item --

In [12]:
#Enter Data Release Abstract between single quotes below

###############################################################
abstract = 'This data release provides digital flight-line and gridded data for a high-resolution airborne magnetic and radiometric survey over the region surrounding the Wet Mountains of southern Colorado, including parts of Custer and Fremont Counties. Data for this survey were collected by Sander Geophysics Limited International (SGL) under contract with the USGS.  The survey was flown in June and July of 2021 using a helicopter equipped with a magnetometer mounted in a stinger extending from the nose of the aircraft and a gamma-ray spectrometer stowed onboard.  The helicopter pilots followed pre-planned flight paths in a grid-like pattern, with east-west lines spaced 150 meters apart and north-south lines spaced 1,000 meters apart.  A terrain clearance of 80 meters above ground could be realized in areas of low relief to maximize detection of gamma-rays, but higher clearances, as much as 200–500 meters, were required over rugged terrain and populated areas for safety reasons.  A total of 17,032 kilometers were flown along the lines, covering a 2,200 square-kilometer irregular area.  SGL performed extensive data processing after completion of flying and delivered the final data and report in February 2022.'
print (abstract)

This data release provides digital flight-line and gridded data for a high-resolution airborne magnetic and radiometric survey over the region surrounding the Wet Mountains of southern Colorado, including parts of Custer and Fremont Counties. Data for this survey were collected by Sander Geophysics Limited International (SGL) under contract with the USGS.  The survey was flown in June and July of 2021 using a helicopter equipped with a magnetometer mounted in a stinger extending from the nose of the aircraft and a gamma-ray spectrometer stowed onboard.  The helicopter pilots followed pre-planned flight paths in a grid-like pattern, with east-west lines spaced 150 meters apart and north-south lines spaced 1,000 meters apart.  A terrain clearance of 80 meters above ground could be realized in areas of low relief to maximize detection of gamma-rays, but higher clearances, as much as 200–500 meters, were required over rugged terrain and populated areas for safety reasons.  A total of 17,03

In [85]:
#Enter Data Release Purpose between single quotes below

#########################################################################################################################
purpose = 'These data were collected as part of the Earth Mapping Resource Initiative (Earth MRI) program of the U.S. Geological Survey (USGS) in collaboration with the Colorado Geological Survey to better understand the geology and critical mineral resources in the area.  Several alkaline intrusive complexes are of particular interest because they are known to be enriched in thorium, rare earth elements (REE), and other critical minerals.\n\nMagnetic surveys measure subtle variations in the Earth’s magnetic field and are sensitive to differences in the magnetic properties of rocks from ground level to many kilometers below.  Radiometric (or gamma-ray) surveys measure naturally occurring low-level radiation and are sensitive to the presence of rocks near the surface that are enriched in uranium, thorium, or potassium.  Together, the two types of survey data provide valuable information for constructing geologic maps that researchers can use to better understand the mineral resources in the region.'
#########################################################################################################################
print (purpose)

These data were collected as part of the Earth Mapping Resource Initiative (Earth MRI) program of the U.S. Geological Survey (USGS) in collaboration with the Colorado Geological Survey to better understand the geology and critical mineral resources in the area.  Several alkaline intrusive complexes are of particular interest because they are known to be enriched in thorium, rare earth elements (REE), and other critical minerals.

Magnetic surveys measure subtle variations in the Earth’s magnetic field and are sensitive to differences in the magnetic properties of rocks from ground level to many kilometers below.  Radiometric (or gamma-ray) surveys measure naturally occurring low-level radiation and are sensitive to the presence of rocks near the surface that are enriched in uranium, thorium, or potassium.  Together, the two types of survey data provide valuable information for constructing geologic maps that researchers can use to better understand the mineral resources in the region.


In [132]:
#login to ScienceBase
sb = pysb.SbSession()
username = input("Username:  ")
sb.loginc(str(username))
#Check to see if login is successful
sb.is_logged_in()
#Get the ScienceBase Login session info
sb.get_session_info()

Username:  pbrown@usgs.gov
········


{'fullDisplayName': 'Philip J Brown II [pbrown@usgs.gov]',
 'isLoggedIn': True,
 'jossoSessionId': '726976D1C0C41D3F1F744AE3015CFBCF',
 'displayName': 'Philip J Brown II',
 'email': 'pbrown@usgs.gov',
 'username': 'pbrown@usgs.gov'}

In [40]:
# Now lets get the item - be sure to use the correct catalog number!!!
# catNum = '62291b66d34ee0c6b38b88ea'
item_json = sb.get_item(catNum)
#print ("Public Item: \n\t" + str(item_json))
print(json.dumps(item_json, indent=2, sort_keys=True))

{
  "body": "Data release of airborne magnetic and radiometric data for Wet Mountains survey, Colorado",
  "browseCategories": [
    "Data Release - In Progress",
    "Data"
  ],
  "browseTypes": [
    "Shapefile",
    "Downloadable",
    "Map Service",
    "OGC WMS Layer",
    "OGC WFS Layer"
  ],
  "contacts": [
    {
      "contactType": "person",
      "email": "tien@usgs.gov",
      "name": "V. J. S Grauch",
      "orcId": "0000-0001-7656-971X",
      "organization": {},
      "primaryLocation": {
        "mailAddress": {},
        "streetAddress": {}
      },
      "type": "Point of Contact"
    },
    {
      "active": true,
      "contactType": "person",
      "email": "ericanderson@usgs.gov",
      "firstName": "Eric",
      "jobTitle": "Research Geophysicist",
      "lastName": "Anderson",
      "middleName": "D",
      "name": "Eric D Anderson",
      "oldPartyId": 211,
      "onlineResource": "https://usgs.gov/staff-profiles/eric-d-anderson",
      "orcId": "0000-0002-0138-

In [60]:
#Get files for file listing below
#Get the names of the file items in ScienceBase:
#print(item_json['files'])

#for item in item_json['distributionLinks']:
 #for files in item['files']:
  #print (item['name'])

for item in item_json['facets']:
 for files in item['files']:
  print (item['name'])

FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
LinePath_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
LinePath_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
LinePath_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
LinePath_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
LinePath_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021
LinePath_

In [14]:
#Enter Data Release Supplementary Information between single quotes below

#########################################################################################################################
supplinf = 'References cited: \n'\
+ '\nDay, W.D., 2019, The Earth MappingResources Initiative (Earth MRI): Mapping the nations critical mineral resources: U.S. Geological Survey Fact Sheet 2019-3007, https://doi.org/10.3133/fs20193007. \n'\
+ '\nFiles that are available in this publication include flight line data for the magnetic survey in an ascii .csv file, flight line data for the radiometric survey in an ascii .csv file, geoTIFF images of the total magnetic intensity and radiometric total count, potassium, thorium, and uranium values, grids of these data, and a report describing the survey parameters, field operations, quality control and data reduction procedures. A zip file containing the contractor\'s deliverable products is also included.  These files are provided as recieved from the contractor. Binary files in this package are readable using the free software \"Geosoft Viewer\"" or commercial software \"Oasis montaj\".' \
+ '\n\nThis metadata describes the following files:'\
+ '\n\tContractorsDeliverables_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.zip'\
+ '\n\tThumbnailMagRTP_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.png'\
+ '\n\tFootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.shp'\
+ '\n\tLinePath_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.shp'\
+ '\n\nThe zip compressed Contactor\'s Deliverables are provided as an original data as recieved archive only.  As such they are not described individually in the Entity and Attribute section of this metadata.  The original contractor\'s report and contractor provided ReadMe files are part of this zip package and contain the entity and attribute descriptions for all these files as received.'\
+ '\nThe Contactor\'s Deliverables zip file package contains the following files:'\
+ '\n\tFvm.grd\n\tFvm.xyz\n\tMag.grd\n\tMag.xyz\n\tREADME-Mag.txt\n\tTer.grd\n\tTer.xyz\n\tTR-899-001.pdf\n\tWet_Mountains_Dose.grd\n\tWet_Mountains_Dose.xyz\n\tWet_Mountains_Mag.xyz\n\tWet_Mountains_Pot.grd\n\tWet_Mountains_Pot.xyz\n\tWet_Mountains_Radiometric.xyz\n\tWet_Mountains_RTK.grd\n\tWet_Mountains_RTK.xyz\n\tWet_Mountains_RUK.grd\n\tWet_Mountains_RUK.xyz\n\tWet_Mountains_RUT.grd\n\tWet_Mountains_RUT.xyz\n\tWet_Mountains_Tho.grd\n\tWet_Mountains_Tho.xyz\n\tWet_Mountains_Ura.grd\n\tWet_Mountains_Ura.xyz'
#########################################################################################################################
print (supplinf)

References cited: 

Day, W.D., 2019, The Earth MappingResources Initiative (Earth MRI): Mapping the nations critical mineral resources: U.S. Geological Survey Fact Sheet 2019-3007, https://doi.org/10.3133/fs20193007. 

Files that are available in this publication include flight line data for the magnetic survey in an ascii .csv file, flight line data for the radiometric survey in an ascii .csv file, geoTIFF images of the total magnetic intensity and radiometric total count, potassium, thorium, and uranium values, grids of these data, and a report describing the survey parameters, field operations, quality control and data reduction procedures. A zip file containing the contractor's deliverable products is also included.  These files are provided as recieved from the contractor. Binary files in this package are readable using the free software "Geosoft Viewer"" or commercial software "Oasis montaj".

This metadata describes the following files:
	ContractorsDeliverables_AirborneMagneticA

In [15]:
#Enter Data Release Begining Date between single quotes below

#########################################################################################################################
begdate = '20210615'
#########################################################################################################################
print (begdate)

20210615


In [16]:
#Enter Data Release Begining Date between single quotes below

#########################################################################################################################
enddate = '20210727'
#########################################################################################################################
print (enddate)

20210727


In [17]:
#Enter Data Release Access Constraints between single quotes below

#########################################################################################################################
accconst = 'None. Please see \'Distribution Info\' for details.'
#########################################################################################################################
print (accconst)

None. Please see 'Distribution Info' for details.


In [18]:
#Enter Data Release Use Constraints between single quotes below

#########################################################################################################################
useconst = 'Any use of trade, firm, or product names is for descriptive purposes only and does not imply endorsement by the U.S. Government.  Although these data have been processed successfully on a computer system at the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data for other purposes, nor on all computer systems, nor shall the act of distribution constitute any such warranty. The USGS or the U.S. Government shall not be held liable for improper or incorrect use of the data described and/or contained herein.'
#########################################################################################################################
print (useconst)

Any use of trade, firm, or product names is for descriptive purposes only and does not imply endorsement by the U.S. Government.  Although these data have been processed successfully on a computer system at the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data for other purposes, nor on all computer systems, nor shall the act of distribution constitute any such warranty. The USGS or the U.S. Government shall not be held liable for improper or incorrect use of the data described and/or contained herein.


In [19]:
#Enter Data Release Data Contact between single quotes below

#########################################################################################################################
cntperDataContact = 'Philip J. Brown'
#########################################################################################################################
print (cntperDataContact)

Philip J. Brown


In [20]:
#Enter Data Release Data Contact's Position between single quotes below

#########################################################################################################################
cntposDataContact = 'Geophysicist'
#########################################################################################################################
print (cntposDataContact)

Geophysicist


In [21]:
#Enter Data Release Data Contact's Phone Number between single quotes below

#########################################################################################################################
cntvoiceDataContact = '303-236-1310'
#########################################################################################################################
print (cntvoiceDataContact)

303-236-1310


In [22]:
#Enter Data Release Data Contact's email between single quotes below

#########################################################################################################################
cntemailDataContact = 'pjbrown@usgs.gov'
#########################################################################################################################
print (cntemailDataContact)

pjbrown@usgs.gov


In [23]:
#Enter Data Release Data Contact's Phone Number between single quotes below

#########################################################################################################################
datacred = 'This survey and release of the dataset was funded by the Earth Mapping Resources Initiative (Earth MRI).'
#########################################################################################################################
print (datacred)

This survey and release of the dataset was funded by the Earth Mapping Resources Initiative (Earth MRI).


In [24]:
#Enter Data Release attraccr between single quotes below

#########################################################################################################################
attraccr = 'Daily quality control of magnetic data was performed by the contractor. Further quality control measures occurred post flight by the contractor and USGS scientist-in-charge before approval and acceptance of the final data. The data have been peer reviewed and compared with related ancillary data.'
#########################################################################################################################
print (attraccr)

Daily quality control of magnetic data was performed by the contractor. Further quality control measures occurred post flight by the contractor and USGS scientist-in-charge before approval and acceptance of the final data. The data have been peer reviewed and compared with related ancillary data.


In [25]:
#Enter Data Release logic between single quotes below

#########################################################################################################################
logic = 'Data were visually inspected for reasonableness and spatial consistency.'
#########################################################################################################################
print (logic)

Data were visually inspected for reasonableness and spatial consistency.


In [26]:
#Enter Data Release complete between single quotes below

#########################################################################################################################
complete = 'Data set is considered complete for the information presented, as described in the abstract. Users are advised to read the rest of the metadata record carefully for additional details.'
#########################################################################################################################
print (complete)

Data set is considered complete for the information presented, as described in the abstract. Users are advised to read the rest of the metadata record carefully for additional details.


In [27]:
#Enter Data Release horizpar between single quotes below

#########################################################################################################################
horizpar = 'Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.'
#########################################################################################################################
print (horizpar)

Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.


In [28]:
#Enter Data Release vertaccr between single quotes below

#########################################################################################################################
vertaccr = 'Field quality control and assurance procedures were undertaken by the Aeoroquest Limited. field crew and are described in the contractors report included with this data release.'
#########################################################################################################################
print (vertaccr)

Field quality control and assurance procedures were undertaken by the Aeoroquest Limited. field crew and are described in the contractors report included with this data release.


In [29]:
#Enter Data Release latres between single quotes below

#########################################################################################################################
latres = '0.000001'
#########################################################################################################################
print (latres)

0.000001


In [30]:
#Enter Data Release Data longres between single quotes below

#########################################################################################################################
longres = '0.000001'
#########################################################################################################################
print (longres)

0.000001


In [31]:
#Enter Data Release horizdn between single quotes below

#########################################################################################################################
horizdn = 'WGS_1984'
#########################################################################################################################
print (horizdn)


WGS_1984


In [32]:
#Enter Data Release ellips between single quotes below

#########################################################################################################################
ellips = 'WGS_1984'
#########################################################################################################################
print (ellips)

WGS_1984


In [33]:
#Enter Data Release semiaxis between single quotes below

#########################################################################################################################
semiaxis = '6378206.4'
#########################################################################################################################
print (semiaxis)


6378206.4


In [34]:
#Enter Data Release denflat between single quotes below

#########################################################################################################################
denflat = '294.9786982'
#########################################################################################################################
print (denflat)

294.9786982


In [35]:
#Enter Data Release Data Contact's Phone Number between single quotes below

#########################################################################################################################
distliab = 'Unless otherwise stated, all data, metadata and related materials are considered to satisfy the quality standards relative to the purpose for which the data were collected. Although these data and associated metadata have been reviewed for accuracy and completeness and approved for release by the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data on any other system or for general or scientific purposes, nor shall the act of distribution constitute any such warranty.'
#########################################################################################################################
print (distliab)

Unless otherwise stated, all data, metadata and related materials are considered to satisfy the quality standards relative to the purpose for which the data were collected. Although these data and associated metadata have been reviewed for accuracy and completeness and approved for release by the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data on any other system or for general or scientific purposes, nor shall the act of distribution constitute any such warranty.


In [108]:
#Enter Data Release metadata creation date between single quotes below

#########################################################################################################################
metd = '20220524'
#########################################################################################################################
print (metd)

20220524


In [37]:
#Enter Data Release cntperMetaInfo between single quotes below

#########################################################################################################################
cntperMetaInfo = 'Philip Brown'
#########################################################################################################################
print (cntperMetaInfo)

Philip Brown


In [38]:
#Enter Data Release cntposMetaInfo between single quotes below

#########################################################################################################################
cntposMetaInfo = 'Geophysicist'
#########################################################################################################################
print (cntposMetaInfo)

Geophysicist


In [39]:
#Enter Data Release cntvoiceMetaInfo between single quotes below

#########################################################################################################################
cntvoiceMetaInfo = '303-236-1310'
#########################################################################################################################
print (cntvoiceMetaInfo)

303-236-1310


In [40]:
#Enter Data Release cntemailMetaInfo between single quotes below

#########################################################################################################################
cntemailMetaInfo = 'pbrown@usgs.gov'
#########################################################################################################################
print (cntemailMetaInfo)

pbrown@usgs.gov


In [41]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeISO = ['geoscientificInformation','economy']
#########################################################################################################################

txtThemeISO = '\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>'
for i in range(len(arrThemeISO)):
 txtThemeISO = txtThemeISO +  '\n\t\t\t\t<themekey>' + arrThemeISO[i] + '</themekey>'

txtThemeISO = txtThemeISO + '\n\t\t\t</theme>'
print(str(txtThemeISO))


			<theme>
				<themekt>ISO 19115 Topic Category</themekt>
				<themekey>geoscientificInformation</themekey>
				<themekey>economy</themekey>
			</theme>


In [42]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeNONE = ['Mineral Resources Program','MRP', 'Geology, Geophysics, and Geochemistry Science Center' \
            'GGGSC','Earth Mapping Resources Initiative','EarthMRI','horizontal magnetic gradiometry', \
            'vertical magnetic gradient','airborne geophysical survey','U.S. Geological Survey','USGS']
#########################################################################################################################

txtThemeNONE = '\n\t\t\t<theme>\n\t\t\t\t<themekt>None</themekt>'
for i in range(len(arrThemeNONE)):
 txtThemeNONE = txtThemeNONE +  '\n\t\t\t\t<themekey>' + arrThemeNONE[i] + '</themekey>'

txtThemeNONE = txtThemeNONE + '\n\t\t\t</theme>'
print(str(txtThemeNONE))


			<theme>
				<themekt>None</themekt>
				<themekey>Mineral Resources Program</themekey>
				<themekey>MRP</themekey>
				<themekey>Geology, Geophysics, and Geochemistry Science CenterGGGSC</themekey>
				<themekey>Earth Mapping Resources Initiative</themekey>
				<themekey>EarthMRI</themekey>
				<themekey>horizontal magnetic gradiometry</themekey>
				<themekey>vertical magnetic gradient</themekey>
				<themekey>airborne geophysical survey</themekey>
				<themekey>U.S. Geological Survey</themekey>
				<themekey>USGS</themekey>
			</theme>


In [43]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeUSGS = ['magnetic field (earth)','geophysics', 'geospatial datasets', \
            'aeromagnetic surveying']
#########################################################################################################################

txtThemeUSGS = '\n\t\t\t<theme>\n\t\t\t\t<themekt>USGS Thesaurus</themekt>'
for i in range(len(arrThemeUSGS)):
 txtThemeUSGS = txtThemeUSGS +  '\n\t\t\t\t<themekey>' + arrThemeUSGS[i] + '</themekey>'

txtThemeUSGS = txtThemeUSGS + '\n\t\t\t</theme>'
print(str(txtThemeUSGS))


			<theme>
				<themekt>USGS Thesaurus</themekt>
				<themekey>magnetic field (earth)</themekey>
				<themekey>geophysics</themekey>
				<themekey>geospatial datasets</themekey>
				<themekey>aeromagnetic surveying</themekey>
			</theme>


In [44]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrPlaceGNIS = ['Fremont County','Custer County', 'Wet Mountains','Rosita Hills','Grape Creek','Canon City','Oak Creek','Wilson Creek','North Hardscrabble Creek','Gorge Hills','Antelope Creek','Grand Canyon Hills','Texas Creek', 'Tallahassee Creek', 'Arkansas River']
#########################################################################################################################

txtPlaceGNIS = '\n\t\t\t<place>\n\t\t\t\t<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov </placekt>'
for i in range(len(arrPlaceGNIS)):
 txtPlaceGNIS = txtPlaceGNIS +  '\n\t\t\t\t<placekey>' + arrPlaceGNIS[i] + '</placekey>'


txtPlaceGNIS = txtPlaceGNIS + '\n\t\t\t</place>'
print(str(txtPlaceGNIS))


			<place>
				<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov </placekt>
				<placekey>Fremont County</placekey>
				<placekey>Custer County</placekey>
				<placekey>Wet Mountains</placekey>
				<placekey>Rosita Hills</placekey>
				<placekey>Grape Creek</placekey>
				<placekey>Canon City</placekey>
				<placekey>Oak Creek</placekey>
				<placekey>Wilson Creek</placekey>
				<placekey>North Hardscrabble Creek</placekey>
				<placekey>Gorge Hills</placekey>
				<placekey>Antelope Creek</placekey>
				<placekey>Grand Canyon Hills</placekey>
				<placekey>Texas Creek</placekey>
				<placekey>Tallahassee Creek</placekey>
				<placekey>Arkansas River</placekey>
			</place>


In [45]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrPlaceCom = ['Colorado', 'United States']
#########################################################################################################################

txtPlaceCom = '\n\t\t\t<place>\n\t\t\t\t<placekt>Common geographic areas</placekt>'
for i in range(len(arrPlaceCom)):
 txtPlaceCom = txtPlaceCom +  '\n\t\t\t\t<placekey>' + arrPlaceCom[i] + '</placekey>'

txtPlaceCom = txtPlaceCom + '\n\t\t\t</place>'
print(str(txtPlaceCom))


			<place>
				<placekt>Common geographic areas</placekt>
				<placekey>Colorado</placekey>
				<placekey>United States</placekey>
			</place>


In [46]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrPlaceNone = ['USA', 'United States of America', 'south-central Colorado']
#########################################################################################################################

txtPlaceNone = '\n\t\t\t<place>\n\t\t\t\t<placekt>None</placekt>'
for i in range(len(arrPlaceNone)):
 txtPlaceNone = txtPlaceNone +  '\n\t\t\t\t<placekey>' + arrPlaceNone[i] + '</placekey>'

txtPlaceNone = txtPlaceNone + '\n\t\t\t</place>'
print(str(txtPlaceNone))


			<place>
				<placekt>None</placekt>
				<placekey>USA</placekey>
				<placekey>United States of America</placekey>
				<placekey>south-central Colorado</placekey>
			</place>


In [47]:
## Create editable keywords example.  
## Example text is created after running this cell
## This text is displayed by running "display(keywords) below
keywords = '\t\t<keywords>\n' + txtThemeUSGS + txtThemeISO + txtThemeNONE + txtPlaceGNIS + txtPlaceCom + txtPlaceNone +'\n\t\t</keywords>'
print (keywords)

		<keywords>

			<theme>
				<themekt>USGS Thesaurus</themekt>
				<themekey>magnetic field (earth)</themekey>
				<themekey>geophysics</themekey>
				<themekey>geospatial datasets</themekey>
				<themekey>aeromagnetic surveying</themekey>
			</theme>			<theme>
				<themekt>ISO 19115 Topic Category</themekt>
				<themekey>geoscientificInformation</themekey>
				<themekey>economy</themekey>
			</theme>
			<theme>
				<themekt>None</themekt>
				<themekey>Mineral Resources Program</themekey>
				<themekey>MRP</themekey>
				<themekey>Geology, Geophysics, and Geochemistry Science CenterGGGSC</themekey>
				<themekey>Earth Mapping Resources Initiative</themekey>
				<themekey>EarthMRI</themekey>
				<themekey>horizontal magnetic gradiometry</themekey>
				<themekey>vertical magnetic gradient</themekey>
				<themekey>airborne geophysical survey</themekey>
				<themekey>U.S. Geological Survey</themekey>
				<themekey>USGS</themekey>
			</theme>
			<place>
				<placekt>USGS Geographic Names I

# Create process descriptions and dates

In [51]:
##### Now let's create the lineage 2-D array of Process steps and dates
#Add text below in single quotes seperated by commas.  Sequencial 2-D arrays are seperated by brackets []
#2-D Array form is [['procdesc1','procdate1'],['procdesc2','procdate2'],etc...]

######################################################################################################################################################################################################################################################################################################################################################################################
arrLineage = [ \
             ['The airborne survey was flown under contract with Sanders Geophysics Limited from June 15, 2021 to July 27, 2021.', '20210721'] \
             ,['The survey data and report were delivered on February 25, 2022.', '20220225'] \
             ,['Created Esri shapefiles of area footprint and line paths for the survey using the line data provided by the contractor.','20220516'] \
             ,['Final magnetic data were exported to CSV-format with a defined null value to meet open data requirements.','20220519'] \
             ,['Created georeferenced rasters of the data grids provided by the contractor.','20220523'] \
             ,['Final radiometric data were exported to netCDF-format with a defined null value to meet open data requirements.','20220525'] \
             ]
######################################################################################################################################################################################################################################################################################################################################################################################

txtLineage = '\t\t<lineage>'
for i in range(len(arrLineage)):
 txtLineage = txtLineage + '\n\t\t\t<procstep>' +  '\n\t\t\t\t<procdesc>' + arrLineage[i][0] + '</procdesc>\n\t\t\t\t' \
    + '<procdate>' + arrLineage[i][1] + '</procdate>' + '\n\t\t\t</procstep>'

txtLineage = txtLineage + '\n\t\t</lineage>'
print(str(txtLineage))
lineage = txtLineage

		<lineage>
			<procstep>
				<procdesc>The airborne survey was flown under contract with Sanders Geophysics Limited from June 15, 2021 to July 27, 2021.</procdesc>
				<procdate>20210721</procdate>
			</procstep>
			<procstep>
				<procdesc>The survey data and report were delivered on February 25, 2022.</procdesc>
				<procdate>20220225</procdate>
			</procstep>
			<procstep>
				<procdesc>Created Esri shapefiles of area footprint and line paths for the survey using the line data provided by the contractor.</procdesc>
				<procdate>20220516</procdate>
			</procstep>
			<procstep>
				<procdesc>Final magnetic data were exported to CSV-format with a defined null value to meet open data requirements.</procdesc>
				<procdate>20220519</procdate>
			</procstep>
			<procstep>
				<procdesc>Created georeferenced rasters of the data grids provided by the contractor.</procdesc>
				<procdate>20220523</procdate>
			</procstep>
			<procstep>
				<procdesc>Final radiometric data were exported to 

# Step One - Set Directory Paths
## Please set directory paths below
### Directory paths include
- Data Path: This is the path to the data, data structure should have a directory for each station
- Template Path: The path to the XML metadata template being used for the data.  This template should already include all information common to all child metadata files e.g. originators, larger work citation, etc.

In [70]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
# The r signifies a string literal. Use for paths.

magMetaDataTemplatePath = r"C:\CurrentWork\DataReleases\IP-139145_WetMountains"
magMetaDataTemplateName = "AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021_MetaData--TEMPLATE.xml"
magCSVfileDir = r"C:\CurrentWork\DataReleases\IP-139145_WetMountains\Uploads\MagChild"
magCSVfileName = "LineData_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.csv"

In [71]:
#Check Paths for the fun of it
magCSVfilePath = magCSVfileDir + '\\' + magCSVfileName
print ('The Mag CSV Data Path is: ' + '"' + magCSVfilePath + '"')#magCSVfilePath
magMetadataTemplatefilePath = magMetaDataTemplatePath + "\\" + magMetaDataTemplateName
print ('The Template File Path is: ' + '"' + magMetadataTemplatefilePath + '"')

The Mag CSV Data Path is: "C:\CurrentWork\DataReleases\IP-139145_WetMountains\Uploads\MagChild\LineData_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.csv"
The Template File Path is: "C:\CurrentWork\DataReleases\IP-139145_WetMountains\AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021_MetaData--TEMPLATE.xml"


In [109]:
tifFilesPath = r"C:\CurrentWork\DataReleases\IP-139145_WetMountains\Uploads\MagChild"
tifList = glob.glob(os.path.join(tifFilesPath, '*.*'),  recursive=True)
fileListing = ''
rspFileListing = []
for i in range(len(tifList)):
    splitTifList = tifList[i].split('\\')
    fileListing = fileListing +  splitTifList[len(splitTifList) - 1]
    rspFileListing.append(splitTifList[len(splitTifList) - 1])
#print (fileListing)
fileListing

'LineData_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.csvRTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tfwRTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tifRTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tif.aux.xmlRTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tif.ovr'

In [110]:
tifFilesPath = r"C:\CurrentWork\DataReleases\IP-139145_WetMountains\Uploads\MagChild"
List = glob.glob(os.path.join(tifFilesPath, '*.tif'),  recursive=True)
fileListing = ''
rspFileListing = []
for i in range(len(tifList)):
    splitTifList = tifList[i].split('\\')
    fileListing = fileListing +  splitTifList[len(splitTifList) - 1] + '\n'
    rspFileListing.append(splitTifList[len(splitTifList) - 1])
#print (fileListing)
rspFileListing

['LineData_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.csv',
 'RTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tfw',
 'RTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tif',
 'RTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tif.aux.xml',
 'RTPMag_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.tif.ovr']

## Lets now import and index values from the CSV file
- We need create arrays of description and unit values for the metadata template.  
- We also want to run stats on the csv columns for the entity and attributes section

In [111]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
print ('!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!')
print ('Reading file into Pandas, this may take several minutes depending on the file size...')

magDataFrame = pd.read_csv(magCSVfilePath, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
magDataFrame

!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!
Reading file into Pandas, this may take several minutes depending on the file size...


C:\Users\pbrown\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\pbrown\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,AMF,DIR,DIURNAL,DOY,DRAPE_N83,FLT,FTIME,GPS_X_N83,GPS_X_W84,GPS_Y_N83,...,MAGLAG,MAGLEV,MAGLEVCOR,MAGMLEV,MAGMLEVCOR,MAGUC,MSL_Z,RADAR,TER,YEAR
0,970.415,W,29.387,166,3388.72,1001,45891.3,489046.68,489045.78,4209068.01,...,51599.437,51560.862,-9.188,51560.141,-0.721,51593.628,3386.35,123.93,3264.41,2021
1,972.556,W,29.386,166,3388.94,1001,45891.4,489043.57,489042.66,4209068.10,...,51601.569,51562.995,-9.188,51562.273,-0.721,51595.526,3386.56,123.02,3265.56,2021
2,974.541,W,29.385,166,3389.16,1001,45891.5,489040.46,489039.55,4209068.20,...,51603.544,51564.970,-9.188,51564.248,-0.722,51597.242,3386.78,122.08,3266.75,2021
3,976.354,W,29.385,166,3389.37,1001,45891.6,489037.35,489036.44,4209068.33,...,51605.349,51566.776,-9.188,51566.054,-0.722,51598.764,3387.01,121.13,3267.96,2021
4,977.986,W,29.384,166,3389.59,1001,45891.7,489034.25,489033.34,4209068.43,...,51606.971,51568.399,-9.188,51567.676,-0.722,51600.078,3387.24,120.15,3269.20,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4754798,-191.389,N,-5.560,166,1975.93,1001,55195.2,489124.62,489123.72,4234031.01,...,50557.259,50557.697,-4.772,50557.668,-0.029,50562.554,1995.21,84.59,1910.70,2021
4754799,-191.119,N,-5.556,166,1975.74,1001,55195.3,489124.43,489123.52,4234034.31,...,50557.569,50557.978,-4.772,50557.959,-0.019,50562.818,1995.03,84.47,1910.65,2021
4754800,-190.849,N,-5.552,166,1975.56,1001,55195.4,489124.23,489123.32,4234037.61,...,50557.884,50558.260,-4.772,50558.251,-0.009,50563.081,1994.85,84.34,1910.60,2021
4754801,-190.582,N,-5.547,166,1975.38,1001,55195.5,489123.99,489123.08,4234040.91,...,50558.201,50558.540,-4.772,50558.537,-0.003,50563.343,1994.66,84.20,1910.56,2021


### EandA Channel Templates for the magnetic data are available at the following link to a shared Jupyter Notebook
https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroMagEntityAndAttributeArchive.ipynb


In [58]:
# Get Min and Max Latitude and Longitude
#---------------------------------------------------------------------------------
#*************!!!MAKE SURE LAT AND LONG CHANNELS ARE CORRECT BELOW!!!*************
#********* ENTER CORRECT LAT/LONG CHANNEL NAMES BETWEEN SINGLE QUOTES ************
#*****!!!Make sure the results are rounded to the correct # of Sig Figs!!!********
#---------------------------------------------------------------------------------

########################################################################################################################
westbc = magDataFrame['LONG'].dropna().max()
########################################################################################################################
westbc = float(westbc)    # convert to float so that number can be rounded 
westbc = round(westbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
eastbc = magDataFrame['LONG'].dropna().min()
#########################################################################################################################
eastbc = float(eastbc)    # convert to float so that number can be rounded  
eastbc = round(eastbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
northbc = magDataFrame['LAT'].dropna().max()
#########################################################################################################################
northbc = float(northbc)    # convert to float so that number can be rounded 
northbc = round(northbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
southbc = magDataFrame['LAT'].dropna().min()
#########################################################################################################################
southbc = float(southbc)    # convert to float so that number can be rounded  
southbc = round(southbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

print ('westbc: ' + str(westbc))
print ('eastbc: ' + str(eastbc))
print ('northbc: ' + str(northbc))
print ('southbc: ' + str(southbc))

westbc: -105.1235
eastbc: -105.628371
northbc: 38.585017
southbc: 38.028616


In [59]:
# now get an array of the header names
magHeaderList = list(magDataFrame.columns.values)
magHeaderString = ''
for i in range(len(magHeaderList)):
 magHeaderString = magHeaderString + magHeaderList[i] + ','

print (magHeaderString)

AMF,DIR,DIURNAL,DOY,DRAPE_N83,FLT,FTIME,GPS_X_N83,GPS_X_W84,GPS_Y_N83,GPS_Y_W84,GPS_Z_N83,GPS_Z_W84,IGRF,IGRF_REF,LASER,LAT,LINE,LONG,MAGCOMP,MAGDC,MAGLAG,MAGLEV,MAGLEVCOR,MAGMLEV,MAGMLEVCOR,MAGUC,MSL_Z,RADAR,TER,YEAR,


In [60]:
#Create Entity and Attribute Text - substitute vales below with definitions from contractors report:
# EandA Channel Templates for the magnetic data are available at the following link to a shared Jupyter Notebook
# https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroMagEntityAndAttributeArchive.ipynb

#Create Entity and Attribute Text FOR MAG:
csvAttributes = ''
csvEntityDescription = []
csvEntityUnit = []
csvMax = []
csvMin = []
#=======================================================================================================================
# Add entity type and detailed tags
# Below is the entity type description for the Down-Raw spectrum waveforms
entAttText = '\n\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + magCSVfileName + '</enttypl>\n' \
  + '\t\t\t\t<enttypd>Table containing attribute information associated with the magnetic data set.</enttypd>' \
  + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>'

##-----------------Start EandA here
csvEntityDescription.append("Line number")#Line
csvEntityUnit.append("none")#Line
#=======================================================================================================================
csvEntityDescription.append("UTM Easting (NAD83, Zone 16N).")#__X
csvEntityUnit.append("meters")#__X
#=======================================================================================================================
csvEntityDescription.append("UTM Northing (NAD83, Zone 16N).")#__Y
csvEntityUnit.append("meters")#__Y
#=======================================================================================================================
csvEntityDescription.append("Micro-leveled, IGRF removed Total Magnetic Intensity.")#_magF
csvEntityUnit.append("nanoteslas")#_magF
#=======================================================================================================================
csvEntityDescription.append("Date")#Date
csvEntityUnit.append("yyyy.mm.dd")#Date
#=======================================================================================================================
csvEntityDescription.append("Base station total magnetic intensity")#Diurnal
csvEntityUnit.append("nanoteslas")#Diurnal
#=======================================================================================================================
csvEntityDescription.append("Digital Terrain Model")#DTM
csvEntityUnit.append("meters")#DTM
#=======================================================================================================================
csvEntityDescription.append("Fiducial.")#Fid
csvEntityUnit.append("none")#Fid
#=======================================================================================================================
csvEntityDescription.append("Flight number.")#Flight
csvEntityUnit.append("none")#Flight
#=======================================================================================================================
csvEntityDescription.append("GPS altitude.")#Galt
csvEntityUnit.append("meters")#Galt
#=======================================================================================================================
csvEntityDescription.append("Horizontal Gradient of TMI.")#Hgrad
csvEntityUnit.append("nanoteslas per meters")#Hgrad
#=======================================================================================================================
csvEntityDescription.append("Horizontal Gradient of TMI (Raw)")#Hgrad_raw
csvEntityUnit.append("nanoteslas")#Hgrad_raw
#=======================================================================================================================
csvEntityDescription.append("IGRF data of Model 2008")#IGRF
csvEntityUnit.append("nanoteslas")#IGRF
#=======================================================================================================================
csvEntityDescription.append("WGS84")#Lat
csvEntityUnit.append("Decimal Degrees")#Lat
#=======================================================================================================================
csvEntityDescription.append("Integer Line number")#Line.1
csvEntityUnit.append("none")#Line.1
#=======================================================================================================================
csvEntityDescription.append("WGS84.")#Lon
csvEntityUnit.append("Decimal Degrees")#Lon
#=======================================================================================================================
csvEntityDescription.append("Total Magnetic Intensity tail sensor (Raw).")#magA
csvEntityUnit.append("nanoteslas")#magA
#=======================================================================================================================
csvEntityDescription.append("Total Magnetic Intensity diurnal corrected.")#MagD
csvEntityUnit.append("nanoteslas")#MagD
#=======================================================================================================================
csvEntityDescription.append("Total Magnetic Intensity diurnal corrected and IGRF removed.")#MagD_Igrf
csvEntityUnit.append("nanoteslas")#MagD_Igrf
#=======================================================================================================================
csvEntityDescription.append("Micro-leveled, IGRF removed Total Magnetic Intensity.")#magF
csvEntityUnit.append("nanoteslas")#magF
#=======================================================================================================================
csvEntityDescription.append("Total Magnetic Intensity left wing sensor (Raw).")#magL
csvEntityUnit.append("nanoteslas")#magL
#=======================================================================================================================
csvEntityDescription.append("Total Magnetic Intensity right wing sensor (Raw).")#magR
csvEntityUnit.append("nanoteslas")#magR
#=======================================================================================================================
csvEntityDescription.append("Radar Altimeter Height.")#Radalt
csvEntityUnit.append("meters")#Radalt
#=======================================================================================================================
csvEntityDescription.append("UTC Time.")#Time
csvEntityUnit.append("seconds")#Time
#=======================================================================================================================
csvEntityDescription.append("UTM Easting (NAD83, Zone 16N).")#X
csvEntityUnit.append("meters")#X
#=======================================================================================================================
csvEntityDescription.append("UTM Northing (NAD83, Zone 16N).")#Y
csvEntityUnit.append("meters")#Y
#=======================================================================================================================
csvEntityDescription.append("UTM Zone 16N.")#Zone
csvEntityUnit.append("none")#Zone

##-----------------End EandA here

#Now get a min and max value for all columns

csvHeaderListLength = len(csvEntityDescription)
#print ('Length of Entity Description Array: ' + str(len(csvEntityDescription)))
#print ('Length of Header List: ' + str(len(magHeaderList)))

for i in range(csvHeaderListLength): #csvHeaderListLength
 print (magHeaderList[i] + ': ' + str(i))
 strCol = magHeaderList[i]
 csvMax.append(magDataFrame[strCol].max())
 csvMin.append(magDataFrame[strCol].min())
 print ('Max. Value ' + magHeaderList[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + magHeaderList[i] + ' :' + str(csvMin[i]))
 print ('Entt Desc. ' + csvEntityDescription[i])
 print ('Att Unit ' + csvEntityUnit[i] + '\n')
 entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + strCol + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + csvEntityDescription[i] + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>Aeroquest Limited</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + csvEntityUnit[i] + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
 entAttText = entAttText + entAtti
    
entAttText = entAttText  + '\n\t\t</detailed>\n' 

csvAttributes = entAttText

print ('\nEntity and Attribute Text:\n' + csvAttributes)

AMF: 0
Max. Value AMF :4688.677
Min. Value AMF :-926.514
Entt Desc. Line number
Att Unit none

DIR: 1
Max. Value DIR :W
Min. Value DIR :E
Entt Desc. UTM Easting (NAD83, Zone 16N).
Att Unit meters

DIURNAL: 2
Max. Value DIURNAL :29.912
Min. Value DIURNAL :-29.485
Entt Desc. UTM Northing (NAD83, Zone 16N).
Att Unit meters

DOY: 3
Max. Value DOY :208
Min. Value DOY :166
Entt Desc. Micro-leveled, IGRF removed Total Magnetic Intensity.
Att Unit nanoteslas

DRAPE_N83: 4
Max. Value DRAPE_N83 :3579.29
Min. Value DRAPE_N83 :1806.75
Entt Desc. Date
Att Unit yyyy.mm.dd

FLT: 5
Max. Value FLT :1040
Min. Value FLT :1001
Entt Desc. Base station total magnetic intensity
Att Unit nanoteslas

FTIME: 6
Max. Value FTIME :81916.9
Min. Value FTIME :44839.2
Entt Desc. Digital Terrain Model
Att Unit meters

GPS_X_N83: 7
Max. Value GPS_X_N83 :489195.18
Min. Value GPS_X_N83 :445004.45
Entt Desc. Fiducial.
Att Unit none

GPS_X_W84: 8
Max. Value GPS_X_W84 :489194.28
Min. Value GPS_X_W84 :445003.54
Entt Desc. Fli

In [91]:
#strip Tabs from string
stripFileListing = fileListing.replace('\t', '')
#Create array of values by splitting on line return \n
arrayFileListing = stripFileListing.splitlines()
#arrayFileListing[0] = 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_TMFI.tif'
arrayFileListing

['ContractorsReport_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.pdf',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.cpg',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.dbf',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.prj',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.sbn',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.sbx',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.shp',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.shp.IGSKACCGLT0021.26344.892.sr.lock',
 'FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.shx',
 'LinePath_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.cpg',
 'LinePath_AirborneMagneticAndRadiometricSurveyWet

# Now lets create Entity and Attributes for the rest of the files that don't require listing the min/max for the domains.

In [92]:
# Create the Entity and Attributes for the other files being released besides the *.CSV data file
bFilesEntAttText = ''
bFilesEandA = ''
for i in range(len(arrayFileListing)):
    if arrayFileListing[i].find('.tif') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Value</attrlabl>' \
            + '\n\t\t\t\t<attrdef>Unique numeric values contained in each raster cell.</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<rdom>\n\t\t\t\t\t\t<rdommin>0</rdommin>\n\t\t\t\t\t\t<rdommax>255</rdommax>'\
            + '\n\t\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n' 
    if arrayFileListing[i].find('.rtf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Rich Text File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</enttypd>' \
            + '\n\t\t\t\t<enttypds>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Rich Text File</attrlabl>\n\t\t\t\t<attrdef>ReadMe file in Rich Text Format (RTF) containing *.CSV file entities, attributes, and processing notes</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    if arrayFileListing[i].find('.gdd') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary data base file in Geosoft (https://www.geosoft.com) Oasis montaj gdb format. For more information on reading gdb files using publicly available Python tools visit, https://github.com/GeosoftInc/gxpy.</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary data base file</attrlabl>\n\t\t\t\t<attrdef>Binary data base file described in contractor report.</attrdef>\n\t\t\t\t<attrdefs>Aeroquest Limited.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>Binary Data Base File described in contractor report.</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.xml') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII XML File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII text XML formatted metadata file describing the data base file of the same name.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII XML formatted text file</attrlabl>\n\t\t\t\t<attrdef>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.gxf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII grid file in Geosoft (https://www.geosoft.com) Grid Exchange Format, GXF (https://www.geosoft.com/media/uploads/resources/technical-notes/gxfr3d9_1.pdf) </enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII Grid File</attrlabl>\n\t\t\t\t<attrdef>ASCII grid file in GXF format described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>Aeroquest Limited.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>ASCII Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    
    if arrayFileListing[i].find('.grd') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary grid file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary grid file</attrlabl>\n\t\t\t\t<attrdef>Binary grid file described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>Binary Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    bFilesEntAttText = bFilesEandA

print (bFilesEntAttText) 


In [93]:
# Run this cell for key word text to edit.  
# One can edit the text in place and these changes will be present in the created metadata.  
# When complete move on to the next step

display(bFilesEntAttText)

''

In [77]:
#Enter the URL for the magnetic data child for this data release between single quotes below
# https://doi.org/:10.5066/P98CW91W
# https://www.sciencebase.gov/catalog/item/624736ded34e21f82762ef69
#########################################################################################################################
childURL = 'https://www.sciencebase.gov/catalog/item/623219c9d34ec9f19eeb418b'
doiURL = 'https://doi.org/10.5066/P9PISI0X'
#########################################################################################################################
resdesc = childURL
print (childURL)

https://www.sciencebase.gov/catalog/item/623219c9d34ec9f19eeb418b


In [101]:
#Enter the png file name and path
#########################################################################################################################
pngFileName = 'AirborneMagneticSurveyMichiganUpperPeninsulaArea2008_TotalFieldAnomaly.png'
pngDirPath = r'C:\CurrentWork\DataReleases\IP-139145_WetMountains\Uploads\Parent'
#########################################################################################################################

pngFilePath = pngDirPath + "\\" + pngFileName
txtListFormatted = []
txtListFormatted.append(pngFileName)#png file

print('pngFilePath\n' + pngFilePath)

pngFilePath
C:\CurrentWork\DataReleases\IP-139145_WetMountains\Uploads\Parent\AirborneMagneticSurveyMichiganUpperPeninsulaArea2008_TotalFieldAnomaly.png


In [102]:
bFilesEntAttText = ''
bFilesEandA = ''
for i in range(len(arrayFileListing)):
    if arrayFileListing[i].find('.zip') != -1:
        bFilesEandA = bFilesEandA + '\t\t<overview>\n\t\t\t<eaover>' + arrayFileListing[i] + ' is a Geosoft (https://www.geosoft.com) data base file in Oasis montaj proprietary binary gdb format. For more information on reading gdb files using publicly available Python tools visit, https://github.com/GeosoftInc/gxpy.' + arrayFileListing[i] + ' is described in the contractors report and has the same channels and data described in detail for the csv file also included with this data release. </eaover> '\
            + '\n\t\t\t<eadetcit>' + doiURL + '</eadetcit>\n\t\t</overview>\n'
    
    if arrayFileListing[i].find('.pdf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<overview>\n\t\t\t<eaover>' + arrayFileListing[i] + ' is an Aeroquest Limited contractor report in Adobe Portable Document Format (PDF).  This report describes an aeromagnetic survey over the Western Upper Peninsula Area, Michigan for Kennecott Eagle Minerals Company.' + arrayFileListing[i] + ' describes the survey parameters, field operations, quality control results, and data reduction procedures used to produce the aeromagnetic data.</eaover> '\
            + '\n\t\t\t<eadetcit>' + doiURL + '</eadetcit>\n\t\t</overview>\n'
    
    if arrayFileListing[i].find('.png') != -1:
        bFilesEandA = bFilesEandA + '\t\t<overview>\n\t\t\t<eaover>' + arrayFileListing[i] + ' is a Geosoft (https://www.geosoft.com) data base file in Oasis montaj proprietary binary gdb format. For more information on reading gdb files using publicly available Python tools visit, https://github.com/GeosoftInc/gxpy.' + arrayFileListing[i] + ' is described in the contractors report and has the same channels and data described in detail for the csv file also included with this data release. </eaover> '\
            + '\n\t\t\t<eadetcit>' + doiURL + '</eadetcit>\n\t\t</overview>\n'
        
    bFilesEntAttText = bFilesEandA
print (bFilesEntAttText) 

		<overview>
			<eaover>ContractorsReport_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.pdf is an Aeroquest Limited contractor report in Adobe Portable Document Format (PDF).  This report describes an aeromagnetic survey over the Western Upper Peninsula Area, Michigan for Kennecott Eagle Minerals Company.ContractorsReport_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.pdf describes the survey parameters, field operations, quality control results, and data reduction procedures used to produce the aeromagnetic data.</eaover> 
			<eadetcit>https://doi.org/10.5066/P9PISI0X</eadetcit>
		</overview>
		<overview>
			<eaover>PLACEHOLDER-ContractorsDeliverables_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.zip is a Geosoft (https://www.geosoft.com) data base file in Oasis montaj proprietary binary gdb format. For more information on reading gdb files using publicly available Python tools visit, https://github.com/Ge

# Now concatenate all the entity and attribute text


In [103]:
eainfo = bFilesEntAttText
#eainfo = csvAttributes + bFilesEntAttText + txtEandA
eainfo

'\t\t<overview>\n\t\t\t<eaover>ContractorsReport_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.pdf is an Aeroquest Limited contractor report in Adobe Portable Document Format (PDF).  This report describes an aeromagnetic survey over the Western Upper Peninsula Area, Michigan for Kennecott Eagle Minerals Company.ContractorsReport_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.pdf describes the survey parameters, field operations, quality control results, and data reduction procedures used to produce the aeromagnetic data.</eaover> \n\t\t\t<eadetcit>https://doi.org/10.5066/P9PISI0X</eadetcit>\n\t\t</overview>\n\t\t<overview>\n\t\t\t<eaover>PLACEHOLDER-ContractorsDeliverables_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.zip is a Geosoft (https://www.geosoft.com) data base file in Oasis montaj proprietary binary gdb format. For more information on reading gdb files using publicly available Python tools visit, 

## Create the stdorder for all the non-extension (tiff) files 

In [104]:
#Change this URL for each child
jpgURL = 'https://www.sciencebase.gov/catalog/item/622bb21ed34ec9f19eea4361'

arrJpgFileSize = []
for i in range(len(arrayFileListing)):
    jpgFilePath = pngDirPath + "\\" + arrayFileListing[i]
# Get file sizes
    jpgFilesize = os.path.getsize(jpgFilePath)
    jpgFilesize = jpgFilesize/1000000 # convert file size from bytes to megabytes
    arrJpgFileSize.append(jpgFilesize)
      

txtStdOrder = '\t\t<stdorder>'
for i in range(len(arrayFileListing)):
 txtStdOrder = txtStdOrder + '\n\t\t\t<digform>' +  '\n\t\t\t\t<digtinfo>' \
    + '\n\t\t\t\t\t<formname>JPG</formname>' \
    + '\n\t\t\t\t\t<formvern>' + '1' + '</formvern>' \
    + '\n\t\t\t\t\t<formspec>' + arrayFileListing[i] + '</formspec>' \
    + '\n\t\t\t\t\t<formcont>J(oint) P(hotographic) E(xperts) Group (JPEG or JPG) raster image of Tok Alaska electromagnetic and magnetic airborne geophysical survey 2014 inverted resistivity models.</formcont>' \
    + '\n\t\t\t\t\t<filedec>' + 'No compression applied </filedec>' \
    + '\n\t\t\t\t\t<transize>' + str(arrJpgFileSize[i]) + '</transize>\n\t\t\t\t' \
    + '</digtinfo>' + '\n\t\t\t<digtopt>' + '\n\t\t\t\t<onlinopt>' + '\n\t\t\t\t\t<computer>' + '\n\t\t\t\t\t\t<networka>' \
    + '\n\t\t\t\t\t\t\t<networkr>' + jpgURL + '</networkr>' + '\n\t\t\t\t\t\t</networka>' \
    + '\n\t\t\t\t\t\t</computer>' + '\n\t\t\t\t</onlinopt>' + '\n\t\t\t</digtopt>\n\t\t</digform>'
    
txtStdOrder = txtStdOrder  + '\n\t\t\t\t\t\t<fees>' + 'none' + '</fees>' + '\n\t\t</stdorder>'

stdorder = txtStdOrder
print(str(stdorder))


		<stdorder>
			<digform>
				<digtinfo>
					<formname>JPG</formname>
					<formvern>1</formvern>
					<formspec>ContractorsReport_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.pdf</formspec>
					<formcont>J(oint) P(hotographic) E(xperts) Group (JPEG or JPG) raster image of Tok Alaska electromagnetic and magnetic airborne geophysical survey 2014 inverted resistivity models.</formcont>
					<filedec>No compression applied </filedec>
					<transize>16.813678</transize>
				</digtinfo>
			<digtopt>
				<onlinopt>
					<computer>
						<networka>
							<networkr>https://www.sciencebase.gov/catalog/item/622bb21ed34ec9f19eea4361</networkr>
						</networka>
						</computer>
				</onlinopt>
			</digtopt>
		</digform>
			<digform>
				<digtinfo>
					<formname>JPG</formname>
					<formvern>1</formvern>
					<formspec>FootPrint_AirborneMagneticAndRadiometricSurveyWetMountainsSouthCentralColorado2021.cpg</formspec>
					<formcont>J(oint) P(hotographic) E(xperts) Gr

# Populate Metadata Template

In [105]:
#Load XML Metadata Template File and Read It
metaData = os.path.join(magMetaDataTemplatePath, magMetaDataTemplateName)
print ('Metadata path: ' + magMetaDataTemplatePath + '\n')
xmlTemplateFile = open(metaData, 'r')
metaDataContent = xmlTemplateFile.readlines()
print(metaDataContent)
xmlTemplateFile.close()


Metadata path: C:\CurrentWork\DataReleases\IP-139145_WetMountains

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '  <idinfo>\n', '    <citation>\n', '      <citeinfo>{origin}\n', '        <pubdate>{pubdate}</pubdate>\n', '        <title>{title}</title>\n', '        <geoform>tabular digital data</geoform>\n', '        <pubinfo>\n', '          <pubplace>Denver, CO</pubplace>\n', '          <publish>U.S. Geological Survey</publish>\n', '        </pubinfo>\n', '        <othercit>{othercit}</othercit>\n', '        <onlink>{onlink}</onlink>\n', '        <lworkcit>\n', '          <citeinfo>\n', '            <origin>Day, W.D.</origin>\n', '            <pubdate>2019</pubdate>\n', '            <title>The Earth MappingResources Initiative (Earth MRI)</title>\n', '            <edition>1.0</edition>\n', '            <serinfo>\n', '              <sername>Mapping the nations critical mineral resources: U.S. Geological Survey Fact Sheet 2019-3007</sername>\n', '              <issue>2019

### !!! Be sure to enter output file name for the metadata below !!!

In [106]:
# Please enter metadata output file name below between the single quotes
#========================================================================================================================
magMetadataOutputFileName = 'AirborneMagneticSurveyMichiganUpperPeninsulaArea2008_Metadata--NEW.xml'
#========================================================================================================================

In [107]:
# Replace values of current metadata template with the appropriate values.  
# All of this input should have been defined when going through the steps outlined above.
lineString = ''
#newMetaDataContent = metaDataContent
myfilename = magMetaDataTemplatePath + "\\" + magMetadataOutputFileName
xmlFile = open(myfilename,"w+")
print('File exported to:\n'+ myfilename)
#print(keywords.value)
for i in range(len(metaDataContent)):
    lineString = metaDataContent[i]
    if lineString.find('{origin}'):
     lineString = lineString.replace('{origin}',origin)
    
    if lineString.find('{pubdate}'):
     lineString = lineString.replace('{pubdate}', pubdate)    
    
    if lineString.find('{title}'):
     lineString = lineString.replace('{title}', title)
    
    if lineString.find('{othercit}'):
     lineString = lineString.replace('{othercit}', othercit)
    
    if lineString.find('{onlink}'):
     lineString = lineString.replace('{onlink}', onlink)
    
    if lineString.find('{abstract}'):
     lineString = lineString.replace('{abstract}', abstract)
    
    if lineString.find('{purpose}'):
     lineString = lineString.replace('{purpose}', purpose)
    
    if lineString.find('{supplinf}'):
     lineString = lineString.replace('{supplinf}', supplinf)
    
    if lineString.find('{begdate}'):
     lineString = lineString.replace('{begdate}', begdate)
    
    if lineString.find('{enddate}'):
     lineString = lineString.replace('{enddate}', enddate)
    
    if lineString.find('{westbc}'):
     lineString = lineString.replace('{westbc}', str(westbc))

    if lineString.find('{eastbc}'):
     lineString = lineString.replace('{eastbc}', str(eastbc))

    if lineString.find('{northbc}'):
     lineString = lineString.replace('{northbc}', str(northbc))
    
    if lineString.find('{southbc}'):
     lineString = lineString.replace('{southbc}', str(southbc))
   
    if lineString.find('{keywords}'):
     lineString = lineString.replace('{keywords}', str(keywords))
    
    if lineString.find('{accconst}'):
     lineString = lineString.replace('{accconst}', accconst)
    
    if lineString.find('{useconst}'):
     lineString = lineString.replace('{useconst}', useconst)
    
    if lineString.find('{cntperDataContact}'):
     lineString = lineString.replace('{cntperDataContact}', cntperDataContact)
    
    if lineString.find('{cntposDataContact}'):
     lineString = lineString.replace('{cntposDataContact}', cntposDataContact)
    
    if lineString.find('{cntvoiceDataContact}'):
     lineString = lineString.replace('{cntvoiceDataContact}', str(cntvoiceDataContact))
    
    if lineString.find('{cntemailDataContact}'):
     lineString = lineString.replace('{cntemailDataContact}', str(cntemailDataContact))
    
    if lineString.find('{datacred}'):
     lineString = lineString.replace('{datacred}', str(datacred))
    
    if lineString.find('{attraccr}'):
     lineString = lineString.replace('{attraccr}', str(attraccr)) 
    
    if lineString.find('{logic}'):
     lineString = lineString.replace('{logic}', logic)
    
    if lineString.find('{complete}'):
     lineString = lineString.replace('{complete}', str(complete))
    
    if lineString.find('{horizpar}'):
     lineString = lineString.replace('{horizpar}', str(horizpar))
    
    if lineString.find('{vertaccr}'):
     lineString = lineString.replace('{vertaccr}', str(vertaccr))
    
    if lineString.find('{lineage}'):
     lineString = lineString.replace('{lineage}', str(lineage))
    
    if lineString.find('{latres}'):
     lineString = lineString.replace('{latres}', str(latres))
    
    if lineString.find('{longres}'):
     lineString = lineString.replace('{longres}', str(longres))
    
    if lineString.find('{horizdn}'):
     lineString = lineString.replace('{horizdn}', str(horizdn))
    
    if lineString.find('{ellips}'):
     lineString = lineString.replace('{ellips}', str(ellips))
    
    if lineString.find('{semiaxis}'):
     lineString = lineString.replace('{semiaxis}', str(semiaxis))
    
    if lineString.find('{denflat}'):
     lineString = lineString.replace('{denflat}', str(denflat))
   
    if lineString.find('{eainfo}'):
     lineString = lineString.replace('{eainfo}', str(eainfo))
    
    if lineString.find('{distliab}'):
     lineString = lineString.replace('{distliab}', str(distliab))
    
    if lineString.find('{metd}'):
     lineString = lineString.replace('{metd}', str(metd))
    
    if lineString.find('{cntperMetaInfo}'):
     lineString = lineString.replace('{cntperMetaInfo}', str(cntperMetaInfo))
    
    if lineString.find('{cntposMetaInfo}'):
     lineString = lineString.replace('{cntposMetaInfo}', str(cntposMetaInfo))
    
    if lineString.find('{cntvoiceMetaInfo}'):
     lineString = lineString.replace('{cntvoiceMetaInfo}', str(cntvoiceMetaInfo))
    
    if lineString.find('{cntemailMetaInfo}'):
     lineString = lineString.replace('{cntemailMetaInfo}', str(cntemailMetaInfo))    
    
    if lineString.find('{stdorder}'):
     lineString = lineString.replace('{stdorder}', str(stdorder))
    
    if lineString.find('{resdesc}'):
     lineString = lineString.replace('{resdesc}', str(resdesc))
    
    else:
     lineString = lineString
    xmlFile.write(lineString)
    
    #print (lineString)
     
    
    
#for r in (metaDataContent):
    #newMetaDataContent = metaDataContent.replace('{title}', drTitle)
    #newMetaDataContent = metaDataContent.replace('{keywords}', keywords.value)
xmlFile.close()

print ('Creation of new metadata file is complete\n\n') 
#Load EDI File and Read It
##checkFile = open(open(myfilename, 'r')
##checkFileContent = checkFile.read()
##checkFile.close()
##print(checkFileContent)

File exported to:
C:\CurrentWork\DataReleases\IP-139145_WetMountains\AirborneMagneticSurveyMichiganUpperPeninsulaArea2008_Metadata--NEW.xml
Creation of new metadata file is complete




### Check this file to see if it is valid against the FGDC metadata standard (FGDC-STD-001-1998)

## https://mrdata.usgs.gov/validation/